# Download

Berikut adalah contoh script untuk mengunduh data GDELT dengan country code ID. Selanjutnya dari csv bisa dipindah ke mysql

In [ ]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])]
print(file_list)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'E:/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')

    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter

        # open the infile and outfile
        with open(infile_name, encoding='utf8', mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

['20181029.export.CSV.zip', '20181028.export.CSV.zip', '20181027.export.CSV.zip', '20181026.export.CSV.zip', '20181025.export.CSV.zip', '20181024.export.CSV.zip', '20181023.export.CSV.zip', '20181022.export.CSV.zip', '20181021.export.CSV.zip', '20181020.export.CSV.zip', '20181019.export.CSV.zip', '20181018.export.CSV.zip', '20181017.export.CSV.zip', '20181016.export.CSV.zip', '20181015.export.CSV.zip', '20181014.export.CSV.zip', '20181013.export.CSV.zip', '20181012.export.CSV.zip', '20181011.export.CSV.zip', '20181010.export.CSV.zip', '20181009.export.CSV.zip', '20181008.export.CSV.zip', '20181007.export.CSV.zip', '20181006.export.CSV.zip', '20181005.export.CSV.zip', '20181004.export.CSV.zip', '20181003.export.CSV.zip', '20181002.export.CSV.zip', '20181001.export.CSV.zip', '20180930.export.CSV.zip', '20180929.export.CSV.zip', '20180928.export.CSV.zip', '20180927.export.CSV.zip', '20180926.export.CSV.zip', '20180925.export.CSV.zip', '20180924.export.CSV.zip', '20180923.export.CSV.zip', 

parsing,
done
20181028.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181027.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181026.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181025.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181024.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181023.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181022.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181021.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181020.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181019.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181018.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181017.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181016.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181015.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181014.export.CSV.zip
downloading, 
extracting,
parsing,
done
20181013.export.CSV.zip
do

# Scraping GDELT
Berikut adalah kode contoh untuk melakukan scraping dengan library newspaper. Skrip ini akan berjalan jika data dari GDELT telah tersimpan ke MySQL

In [ ]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}

with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') LIMIT "+str(offset)+","+str(limit)
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

Berikut adalah contoh untuk melakukan pickling data GDELT

In [8]:
import glob
import pandas as pd

local_path = 'E:/GDELT/'
fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel('CSV.header.fieldids.xlsx', sheet_name='Sheet1',
                         index_col='Column ID', usecols=1)['Field Name']

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print(active_file)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))
    break

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)

E:/GDELT/country\ID0000.tsv


In [13]:
DF

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
798201175,20181029,201810,2018,2018.8192,IDN,INDONESIA,IDN,NaN,NaN,NaN,...,NaN,1,Indonesia,ID,ID,-5,120,ID,20181029,https://www.straitstimes.com/singapore/final-g...
798201178,20181029,201810,2018,2018.8192,IDN,JAKARTA,IDN,NaN,NaN,NaN,...,-797156,4,"Gaza, Israel (general), Israel",IS,IS00,31.4167,34.3333,-797156,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201179,20181029,201810,2018,2018.8192,IDN,JAKARTA,IDN,NaN,NaN,NaN,...,-2696836,4,"Sidoarjo, Jawa Timur, Indonesia",ID,ID08,-7.4478,112.718,-2696836,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201180,20181029,201810,2018,2018.8192,IDN,JAKARTA,IDN,NaN,NaN,NaN,...,-2696836,4,"Gaza, Israel (general), Israel",IS,IS00,31.4167,34.3333,-797156,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201182,20181029,201810,2018,2018.8192,IDNHLH,INDONESIA,IDN,NaN,NaN,NaN,...,-797156,4,"Sidoarjo, Jawa Timur, Indonesia",ID,ID08,-7.4478,112.718,-2696836,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201261,20181029,201810,2018,2018.8192,ISR,ISRAELI,ISR,NaN,NaN,NaN,...,-2696836,4,"Sidoarjo, Jawa Timur, Indonesia",ID,ID08,-7.4478,112.718,-2696836,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201262,20181029,201810,2018,2018.8192,ISR,ISRAELI,ISR,NaN,NaN,NaN,...,-2696836,4,"Gaza, Israel (general), Israel",IS,IS00,31.4167,34.3333,-797156,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201263,20181029,201810,2018,2018.8192,ISR,ISRAEL,ISR,NaN,NaN,NaN,...,-2696836,4,"Jerusalem, Israel (general), Israel",IS,IS00,31.7667,35.2333,-797092,20181029,https://www.orlandoecho.com/news/258207362/ind...
798201264,20181029,201810,2018,2018.8192,ISR,ISRAEL,ISR,NaN,NaN,NaN,...,-2696836,4,"Gaza, Israel (general), Israel",IS,IS00,31.4167,34.3333,-797156,20181029,https://www.orlandoecho.com/news/258207362/ind...


In [14]:
DF.SOURCEURL

GLOBALEVENTID
798201175    https://www.straitstimes.com/singapore/final-g...
798201178    https://www.orlandoecho.com/news/258207362/ind...
798201179    https://www.orlandoecho.com/news/258207362/ind...
798201180    https://www.orlandoecho.com/news/258207362/ind...
798201182    https://www.orlandoecho.com/news/258207362/ind...
798201261    https://www.orlandoecho.com/news/258207362/ind...
798201262    https://www.orlandoecho.com/news/258207362/ind...
798201263    https://www.orlandoecho.com/news/258207362/ind...
798201264    https://www.orlandoecho.com/news/258207362/ind...
798201701    https://www.afr.com/news/tony-abbott-says-naur...
798201839    https://www.afr.com/news/tony-abbott-says-naur...
798201863    https://www.afr.com/news/tony-abbott-says-naur...
798201864    https://www.afr.com/news/tony-abbott-says-naur...
798201871    https://www.afr.com/news/tony-abbott-says-naur...
798208640    https://en.antaranews.com/news/119948/palm-oil...
798208949    https://en.antaranews.com/ne

In [11]:
DF.SQLDATE.count()

3061